In [ ]:
from collections import defaultdict
import numpy as np
from collections import defaultdict
import random
def evaluate_precision(df_recommendations, df_interactions, k=10, max_users=1000):
    user_books = defaultdict(set)
    for _, row in df_interactions.iterrows():
        user_books[row['user_id']].add(row['book_id'])

    total_hits = 0
    total_preds = 0
    total_truth = 0

    for book_id in df_recommendations['input_book_id'].unique():
        recommended_books = df_recommendations[
            df_recommendations['input_book_id'] == book_id
        ]['recommended_book_id'].tolist()[:k]

        users = [user for user, books in user_books.items() if book_id in books]
        if len(users) > max_users:
            users = random.sample(users, max_users)

        for user in users:
            books = user_books[user] - {book_id}
            hits = sum([1 for r in recommended_books if r in books])
            total_hits += hits
            total_preds += k

    precision = total_hits / total_preds if total_preds > 0 else 0
    return precision

def evaluate_triplet_loss(sim_df, interaction_df, margin=0.1, max_users=1000):
    user_groups = interaction_df[interaction_df["is_read"] == True].groupby("user_id")

    all_users = list(user_groups.groups.keys())
    if len(all_users) > max_users:
        sampled_users = set(random.sample(all_users, max_users))
    else:
        sampled_users = set(all_users)

    triplet_losses = []

    sim_dict = sim_df.groupby("input_book_id").apply(
        lambda df: dict(zip(df["recommended_book_id"], df["similarity"]))
    ).to_dict()
    for user, group in user_groups:
        if user not in sampled_users:
            continue
        read_books = list(set(group["book_id"]))
        if len(read_books) < 2:
            continue
        for anchor in read_books:
            positives = [b for b in read_books if b != anchor]
            all_candidates = sim_dict.get(anchor, {})
            for pos in positives:
                sim_ap = all_candidates.get(pos)
                if sim_ap is None:
                    continue
                for neg, sim_an in all_candidates.items():
                    if neg in read_books:
                        continue
                    loss = max(0.0, sim_an - sim_ap + margin)
                    triplet_losses.append(loss)

    return sum(triplet_losses) / len(triplet_losses) if triplet_losses else None



def improvement_score(metric_sbert, metric_collab, higher_is_better=True, eps=1e-8):
    if higher_is_better:
        return (metric_sbert - metric_collab) / (abs(metric_collab) + eps)
    else:
        return (metric_collab - metric_sbert) / (abs(metric_collab) + eps)

def evaluate_all_models(sbert_df, collab_df, interaction_df, k=10):
    # Precision
    precision_sbert = evaluate_precision(sbert_df, interaction_df, k)
    precision_collab = evaluate_precision(collab_df, interaction_df, k)

    # Triplet Loss
    triplet_sbert = evaluate_triplet_loss(sbert_df, interaction_df)
    triplet_collab = evaluate_triplet_loss(collab_df, interaction_df)

    result = {
        "Precision@{}".format(k): {
            "sbert": precision_sbert,
            "collab": precision_collab,
            "improvement": improvement_score(precision_sbert, precision_collab, True)
        },
        "TripletLoss": {
            "sbert": triplet_sbert,
            "collab": triplet_collab,
            "improvement": improvement_score(triplet_sbert, triplet_collab, False)
        }
    }

    return result


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import defaultdict
import random
import os
import gzip
import json

df_collab = pd.read_csv('/content/drive/MyDrive/Capstone/dataset/recommendercollab.csv')
df_sbert = pd.read_csv('/content/drive/MyDrive/Capstone/output_batches/recommender_all_dedup.csv')
df_interactions = pd.read_csv('/content/drive/MyDrive/Capstone/dataset/interaction_part.csv')
df_interactions.columns = df_interactions.columns.str.lower()




<ipython-input-6-f64530e686a1>:10: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_interactions = pd.read_csv('/content/drive/MyDrive/Capstone/dataset/interaction_part.csv')


In [ ]:
results = evaluate_all_models(df_sbert, df_collab, df_interactions, k=10)

print("=== Model Comparison ===")
for metric, values in results.items():
    print(f"{metric}:")
    print(f"  SBERT   : {values['sbert']:.4f}")
    print(f"  Collab  : {values['collab']:.4f}")
    print(f"  Improvement: {values['improvement']:.2%}")


<ipython-input-8-b8ace87888a3>:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sim_dict = sim_df.groupby("input_book_id").apply(
<ipython-input-8-b8ace87888a3>:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sim_dict = sim_df.groupby("input_book_id").apply(


=== Model Comparison ===
Precision@10:
  SBERT   : 0.2127
  Collab  : 0.4056
  Improvement: -47.55%
TripletLoss:
  SBERT   : 0.0953
  Collab  : 0.0765
  Improvement: -24.61%
